In [ ]:
# %load ../../imports.py
"""
These is the standard setup for the notebooks.
"""

%matplotlib inline
%load_ext autoreload
%autoreload 2

#jtplot.style(theme='onedork', context='notebook', ticks=True, grid=False)

import warnings
warnings.filterwarnings("ignore")
from pylab import rcParams
#rcParams['figure.figsize'] = 15, 8
rcParams['figure.figsize'] = 10, 6

import pandas as pd
pd.options.display.max_rows = 999
pd.options.display.max_columns = 999
pd.set_option("display.max_columns", None)
import numpy as np
import os
import matplotlib.pyplot as plt
plt.style.use('paper')

#import data
import copy
from rolldecay.bis_system import BisSystem
from rolldecay import database
from mdldb.tables import Run

from sklearn.pipeline import Pipeline
from rolldecayestimators.transformers import CutTransformer, LowpassFilterDerivatorTransformer, ScaleFactorTransformer, OffsetTransformer
from rolldecayestimators.direct_estimator_cubic import EstimatorQuadraticB, EstimatorCubic
from rolldecayestimators.ikeda_estimator import IkedaQuadraticEstimator
import rolldecayestimators.equations as equations
import rolldecayestimators.lambdas as lambdas
from rolldecayestimators.substitute_dynamic_symbols import lambdify
import rolldecayestimators.symbols as symbols
import sympy as sp

from sklearn.metrics import r2_score



In [ ]:
import matplotlib
print(matplotlib.matplotlib_fname())

In [ ]:
from collections import OrderedDict
from rolldecayestimators.simplified_ikeda_class import SimplifiedIkeda, SimplifiedIkedaABS
from rolldecayestimators.simplified_ikeda import limits_kawahara
import rolldecay.paper_writing as paper_writing

In [ ]:
limits_kawahara

In [ ]:
db = database.get_db()

sql =  """
SELECT * from
loading_conditions
INNER JOIN models
    ON loading_conditions.model_number==models.model_number
    INNER JOIN ships
        ON ships.name == models.ship_name
"""
df_loading_conditions = pd.read_sql_query(sql=sql, con=db.engine)

In [ ]:
loading_condition = df_loading_conditions.median()
#loading_condition = df_loading_conditions.mean()


#df_loading_conditions = df_loading_conditions.select_dtypes(include=np.number)
#loading_condition = (df_loading_conditions.max() - df_loading_conditions.min())/2 + df_loading_conditions.min()

In [ ]:
fi_a=np.deg2rad(10)
beam=loading_condition.beam
lpp=loading_condition.lpp
kg=loading_condition.kg
volume=loading_condition.Volume
draught=loading_condition.TA
A0=loading_condition.A0
lBK=loading_condition.BKL
bBK=loading_condition.BKB
scale_factor=loading_condition.scale_factor

In [ ]:
input = pd.Series()
input['g']=9.81
input['omega_hat']=np.mean(limits_kawahara['OMEGA_hat'])
input['fi_a']=fi_a
input['beam']=beam/scale_factor
input['lpp']=lpp/scale_factor
input['kg']=kg/scale_factor
input['volume']=volume/(scale_factor**3)
input['draught']=draught/scale_factor
input['A0']=A0
input['lBK']=lBK/scale_factor
input['bBK']=bBK/scale_factor

In [ ]:
input['w']= lambdas.omega(beam=input['beam'], g=input['g'], omega_hat=input['omega_hat'])
input['Fn']=0.1
input['V'] = input['Fn']*np.sqrt(input['lpp']*input['g'])

In [ ]:
N = 100
data = np.tile(input.values,(N,1))
inputs_raw = pd.DataFrame(data=data, columns=input.index)

In [ ]:
equations.B_1_hat_equation

In [ ]:
def calculate(inputs, IkedaClass=SimplifiedIkeda):
    si = IkedaClass(**inputs)   
    output = inputs.copy()
    output['B_44_hat'] = si.calculate_B44()
    output['B_W0'] =si.calculate_B_W0()
    output['B_W'] =si.calculate_B_W()
    output['B_F'] =si.calculate_B_F()
    output['B_E'] =si.calculate_B_E()
    output['B_BK'] =si.calculate_B_BK()
    output['B_L'] =si.calculate_B_L()
    output['Bw_div_Bw0'] =si.calculate_Bw_div_Bw0()
    
    return output

In [ ]:
Fns=np.linspace(0,0.2,5)
fig,ax=plt.subplots()

for Fn in Fns:

    inputs = inputs_raw.copy()
    inputs['fi_a']=np.deg2rad(np.linspace(1,15,N))
    inputs['V']=Fn*np.sqrt(inputs['lpp']*inputs['g'])
    
    output = calculate(inputs=inputs)
    output['B_44_hat']-=output.iloc[0]['B_44_hat']
    
    output.plot(x='fi_a', y='B_44_hat',label='Fn=%0.2f'%Fn, ax=ax);
    ax.set_xlabel(r'$\phi_a$')
    ax.set_ylabel(r'$\hat{B_{44}}$')

In [ ]:
inputs = inputs_raw.copy()
inputs['Fn']=np.linspace(0,0.1,N)
inputs['V']=inputs['Fn']*np.sqrt(inputs['lpp']*inputs['g'])

output = calculate(inputs=inputs)

#fig,ax=plt.subplots()
#output.plot(x='Fn', y='B_44_hat', label=r'$\hat{B_{44}}$', ax=ax);
#
#fig,ax=plt.subplots()
#output.plot.area(x='Fn', y = ['B_BK','B_F','B_E','B_L','B_W',], ax=ax)
#
#fig,ax=plt.subplots()
#output.plot(x='Fn', y='Bw_div_Bw0', ax=ax);
#ax.grid(True)
#ax.set_xlabel(r'$F_n$');

In [ ]:
#inputs = inputs_raw.copy()
#x='OG/d'
#inputs[x]=np.linspace(limits_kawahara[x][0],limits_kawahara[x][1],N)
#inputs['OG'] = inputs['OG/d']*inputs['draught']
#inputs['kg'] = -inputs['OG'] + inputs['draught'] 
#
#output = calculate(inputs=inputs)
#
#fig,ax=plt.subplots()
#output.plot.area(x=x, y = ['B_BK','B_F','B_E','B_L','B_W',], ax=ax)

In [ ]:
#inputs = inputs_raw.copy()
#x='fi_a'
#inputs[x]=np.deg2rad(np.linspace(1,15,N))
#
#output = calculate(inputs=inputs)
#
#fig,ax=plt.subplots()
#output.plot.area(x=x, y = ['B_BK','B_F','B_E','B_L','B_W',], ax=ax)

In [ ]:
df_rolldecay = database.load(rolldecay_table_name='rolldecay_quadratic_b', limit_score=0.99, 
                             exclude_table_name='rolldecay_exclude')

df_rolldecay_raw = df_rolldecay.copy()

scale_factor=df_rolldecay['scale_factor']
df_rolldecay['lpp']/=scale_factor
df_rolldecay['TA']/=scale_factor 
df_rolldecay['TF']/=scale_factor
df_rolldecay['draught']=(df_rolldecay['TA'] + df_rolldecay['TF'])/2
df_rolldecay['beam']/=scale_factor
df_rolldecay['BKL']/=scale_factor
df_rolldecay['BKB']/=scale_factor
#df_rolldecay['A0']=db_run.loading_condition.A0
df_rolldecay['kg']/=scale_factor
df_rolldecay['Volume']/=(scale_factor**3)
df_rolldecay['gm']/=scale_factor
df_rolldecay['V']=df_rolldecay['ship_speed']*1.852/3.6/np.sqrt(scale_factor)  #[m/s]
df_rolldecay['rho']=rho=1000
df_rolldecay['g']=g=9.81

df_rolldecay['Fn']=df_rolldecay['V']/np.sqrt(df_rolldecay['lpp']*df_rolldecay['g'])
df_rolldecay['CB']=df_rolldecay['Volume']/(df_rolldecay['lpp']*df_rolldecay['draught']*df_rolldecay['beam'])
df_rolldecay['B/d']=df_rolldecay['beam']/df_rolldecay['draught']
df_rolldecay['OG']=-(df_rolldecay['kg']-df_rolldecay['draught'])
df_rolldecay[r'OG/d']=df_rolldecay['OG']/df_rolldecay['draught']
df_rolldecay['CMID']=df_rolldecay['A0']
df_rolldecay[r'bBk/B']=df_rolldecay['BKB']/df_rolldecay['beam']
df_rolldecay[r'lBk/LPP']=df_rolldecay['BKL']/df_rolldecay['lpp']
df_rolldecay['OMEGA_hat']=lambdas.omega_hat(beam=df_rolldecay.beam, g=df_rolldecay['g'], 
                                            omega0=df_rolldecay['omega0'])


In [ ]:
def CB_changer(df, limits):
    values = np.linspace(limits[0], limits[1], len(df))
    df['CB']=values
    df['volume']=df['CB']*df['lpp']*df['beam']*df['draught']
    return df

def BD_changer(df, limits):
    values = np.linspace(limits[0], limits[1], len(df))
    df[r'B/d']=values
    df['beam']=df[r'B/d']*df['draught']
    return df

def OGd_changer(df, limits):
    values = np.linspace(limits[0], limits[1], len(df))
    df[r'OG/d']=values
    df['OG']=df[r'OG/d']*df['draught']
    df['kg']=-df['OG']+df['draught']
    return df

def CMIDchanger(df, limits):
    values = np.linspace(limits[0], limits[1], len(df))
    df[r'A0']=df[r'CMID']=values
    return df

def bBK_changer(df, limits):
    values = np.linspace(limits[0], limits[1], len(df))
    df[r'bBk/B']=values
    df['bBK']=df[r'bBk/B']*df['beam']
    return df

def lBK_changer(df, limits):
    values = np.linspace(limits[0], limits[1], len(df))
    df[r'lBk/LPP']=values
    df['lBK']=df[r'lBk/LPP']*df['lpp']
    return df

def omega_hat_changer(df, limits):
    values = np.linspace(limits[0], limits[1], len(df))
    df['OMEGA_hat']=values
    df['w']=lambdas.omega(beam=input['beam'], g=input['g'], omega_hat=df['OMEGA_hat'])
    return df

def phi_changer(df, limits):
    values = np.linspace(limits[0], limits[1], len(df))
    df[r'fi_a']=np.deg2rad(values)
    return df

def fn_changer(df, limits):
    values = np.linspace(limits[0], limits[1], len(df))
    df[r'Fn']=values
    df['V']=df['Fn']*np.sqrt(df['lpp']*df['g'])
    return df

changers = OrderedDict() 
changers['CB'] = CB_changer
changers[r'B/d'] = BD_changer
changers[r'OG/d'] = OGd_changer
changers['CMID'] = CMIDchanger
#changers[r'bBk/B'] = bBK_changer
#changers[r'lBk/LPP'] = lBK_changer
changers[r'OMEGA_hat'] = omega_hat_changer
#changers[r'fi_a'] = phi_changer
changers[r'Fn'] = fn_changer

labels={
'CB' : r'$C_b$ [-]',
r'B/d' : r'$\frac{beam}{T}$ [-]',
r'OG/d' : r'$\frac{\overline{OG}}{T}$ [-]',
'CMID' : r'$A_{0}$ [-]',
r'bBk/B' : r'$\frac{BK_B}{beam}$ [-]',
r'lBk/LPP' : r'$\frac{BK_L}{L_{pp}}$ [-]',
r'OMEGA_hat' : r'$\hat{\omega}$ [-]',
r'fi_a' : r'$\phi_a$ [rad]',
r'Fn' : r'$F_n$ [-]',
    
}

limits={key:(df_rolldecay[key].min(),df_rolldecay[key].max()) for key in changers.keys() if key in df_rolldecay}
limits['fi_a'] = (1,10)
limits['Fn'] = (0,0.25)


In [ ]:
#reference_ship = s = input.copy()
#s['CB'] = s['volume']/(s['lpp']*s['beam']*s['draught'])
#s[r'B/d'] = s['beam']/s['draught']
#s[r'OG'] =s['draught']-s['kg']
#s[r'OG/d'] = s['OG']/s['draught']
#s['CMID'] = s['A0']
#s[r'bBk/B'] = s['bBK']/s['lpp']
#s[r'lBk/LPP'] = s['lBK']/s['lpp']
#s[r'OMEGA_hat'] = lambdas.omega_hat(beam=s['beam'], g=g, omega0=s['w'])
#s[r'fi_a'] = fi_a
#s[r'Fn'] = s['Fn']

In [ ]:
#reference_ship = {key:df_rolldecay[key].mean() for key in changers.keys() if key in df_rolldecay}
reference_ship = {key:np.mean(limits_kawahara[key]) for key in changers.keys() if key in limits_kawahara}

reference_ship['fi_a'] = np.mean(np.deg2rad((1,10)))
reference_ship['Fn'] = np.mean((0,0.25))

In [ ]:
N = 100
reference_ship=pd.Series(reference_ship)
data = np.tile(reference_ship.values,(N,1))
inputs_raw = pd.DataFrame(data=data, columns=reference_ship.index)
inputs_raw['lpp']=df_rolldecay['lpp'].mean()
inputs_raw['beam']=df_rolldecay['beam'].mean()
inputs_raw['draught']=df_rolldecay['draught'].mean()
inputs_raw['g']=g
inputs_raw['rho']=rho
#df_rolldecay['Cb']=df_rolldecay['Volume']/(df_rolldecay['lpp']*df_rolldecay['beam']*df_rolldecay['draught'])
#inputs_raw['Cb']=df_rolldecay['Cb'].mean()
#inputs_raw['volume']=inputs_raw['Cb']*inputs_raw['lpp']*inputs_raw['beam']*inputs_raw['draught']


for key, changer in changers.items():
    value = reference_ship[key]
    changer(inputs_raw,(value,value))


In [ ]:
import matplotlib.patches as patches

fig = plt.figure()
fig.set_size_inches(7,7)

ncols=3
#nrows=2*3
nrows=2*2
left_pad=0.09
right_pad=0.01
top_pad=0.12
bottom_pad=0.07

top_padders = [0.0, 0.00]
top_pads = np.tile(top_padders,(1,int(nrows/2)))[0]

bottom_padders = [0.0, 0.05]
bottom_pads = np.tile(top_padders,(1,int(nrows/2)))[0]

left_padders = np.ones(ncols)*0.02
right_padders = 0.0

dX=1/ncols
dx=(1-left_pad-right_pad)/ncols
dx_prims = dx - left_padders - right_padders
xs = np.arange(0,ncols)*(dx) + left_pad + left_padders

y_ratios=np.array([1.0,4.0])
dys=np.tile(y_ratios,(1,int(nrows/2)))[0]
dys/=np.sum(dys)
dys*=(1-bottom_pad-top_pad)
dys-=top_pads - bottom_pads


ys = np.cumsum(dys + top_pads + bottom_pads)
ys = np.concatenate(([0],ys[:-1])) + bottom_pad
ys = np.flipud(ys)
dys = np.flipud(dys)


axes=[]
for y,dy in zip(ys,dys):
    row=[]
    for x,dx_prim in zip(xs,dx_prims):
        ax = fig.add_axes([x,y,dx_prim,dy])
        #ax.get_xaxis().set_visible(False)
        #ax.get_yaxis().set_visible(False)
        row.append(ax)
    axes.append(row)

ylims=[]
ylims1=[]
#axes=axes.flatten()
ylim=0.045
for i,(x, changer) in enumerate(changers.items()):
    
    
    # Add model test results:
    row=i//ncols
    col=i%ncols
    ax1=axes[row*2+1][col]
    
    if x in df_rolldecay:
        df_rolldecay.hist(x,ax=ax1, bins=20)
            
    ax1.set_title(None)
    ax1.grid(False)
    ylims1.append(ax1.get_ylim()[1])
    xlabel=labels[x]
    ax1.set_xlabel(xlabel)
        
    
    ax=axes[row*2][col]
    lims=limits[x]
        
    inputs = inputs_raw.copy()
    inputs = changer(df=inputs, limits=lims)
    output = calculate(inputs=inputs, IkedaClass=SimplifiedIkedaABS)
    
    #lims_extrapolation=limits_extrapolation[x]
    #inputs_extrapolation = inputs_raw.copy()
    #inputs_extrapolation = changer(df=inputs_extrapolation, limits=lims_extrapolation)
    #output_extrapolation = calculate(inputs=inputs_extrapolation, IkedaClass=SimplifiedIkedaABS)
    
    labels_ = [r'$B_{BK}$',r'$B_F$',r'$B_E$',r'$B_L$',r'$B_W$']
    
    output.plot.area(x=x, y = ['B_BK','B_F','B_E','B_L','B_W',], label=labels_, ax=ax)
    if x in limits_kawahara:
        lim_=limits_kawahara[x]
        
        ax.plot([lim_[0],lim_[0]],[0,ylim],'r', label='exceeding limit')
        rect = patches.Rectangle((lim_[0],0),ax.get_xlim()[0]-lim_[0],ylim,linewidth=1,edgecolor='r',facecolor='r', alpha=0.25, hatch='\\')
        ax.add_patch(rect)
        
        ax.plot([lim_[1],lim_[1]],[0,ylim],'r')
        rect = patches.Rectangle((lim_[1],0),ax.get_xlim()[1]-lim_[1],ylim,linewidth=1,edgecolor='r',facecolor='r', alpha=0.25, hatch='\\')
        ax.add_patch(rect)
        
    ax.plot([reference_ship[x], reference_ship[x]],[0,ylim], 'k--', label='reference ship')
        
    
    ax.get_xaxis().set_visible(False)
    
    #ax.grid()
    ax.get_legend().remove()
    ylims.append(ax.get_ylim()[1])
    
    if x in df_rolldecay:
        ax.set_xlim(ax1.get_xlim())
    else:
        ax1.set_xlim(ax.get_xlim())
    
    if col>0:
        ax.get_yaxis().set_visible(False)
        ax1.get_yaxis().set_visible(False)
    else:
        ax.set_ylabel(r'$\hat{B_{44}}$')
        ax1.set_ylabel(r'ships')
        
    #ax.spines['top'].set_visible(False)
    #ax.spines['right'].set_visible(False)
    ax.spines['bottom'].set_visible(False)
    #ax.spines['left'].set_visible(False)
    ax.set_ylim(0,ylim)  # hard coded
    
    ax1.spines['top'].set_visible(False)
    #ax1.spines['right'].set_visible(False)
    #ax1.spines['bottom'].set_visible(False)
    #ax1.spines['left'].set_visible(False)
    
    
#for i,(x, changer) in enumerate(changers.items()):
#    row=i//ncols
#    col=i%ncols
#    ax=axes[row*2][col]
#    ax.set_ylim(0,np.max(ylims))
    
for i,(x, changer) in enumerate(changers.items()):
    row=i//ncols
    col=i%ncols
    ax=axes[row*2+1][col]
    ax.set_ylim(0,np.max(ylims1))
    
#plt.tight_layout()
axes[0][2].legend();

paper_writing.save_fig(fig=fig, name='SI-sensitivity')

In [ ]:
xs

In [ ]:
3*dX

In [ ]:
3*dx

In [ ]:
np.sum(dx_prims)

In [ ]:
fig, ax = plt.subplots()
x = np.arange(0, 4 * np.pi, 0.01)
y = np.sin(x)
ax.plot(x, y, color='black')

threshold = 0.75
ax.axhline(threshold, color='green', lw=2, alpha=0.7)
ax.fill_between(x, -3, 2, where=y > threshold,
                color='green', alpha=0.5, transform=ax.get_xaxis_transform())

In [ ]:
ys